In [2]:
import pandas as pd
import numpy as np

In [23]:
df_val = pd.read_csv('../nss_capstone/df_22_23.csv').drop(columns = ['Unnamed: 0', 'Rot']).fillna('')

In [24]:
df_val['game_id'] = (df_val.index // 2) + 1
df_val.insert(0, 'game_id', df_val.pop('game_id'))

In [25]:
df_val['ML'] = pd.to_numeric(df_val['ML'], errors='coerce')

df_val['Open Spread'] = pd.to_numeric(df_val['Open Spread'], errors='coerce')
df_val['Close Spread'] = pd.to_numeric(df_val['Close Spread'], errors='coerce')

if (df_val['ML'] < 0).any():
    # Update Open Spread and Close Spread values to negative for negative ML rows
    df_val.loc[df_val['ML'] < -1, ['Open Spread', 'Close Spread']] *= -1

In [33]:
# Moves favorites to first row

for game_id in df_val['game_id'].unique():
    game_rows = df_val[df_val['game_id'] == game_id]
    
    if len(game_rows) == 2:
        if game_rows.iloc[0]['Open Spread'] < 0:
            df_val.loc[game_rows.index[1], 'Open Spread'] = np.abs(df_val.loc[game_rows.index[0], 'Open Spread'])
            
        if game_rows.iloc[0]['Close Spread'] < 0:
            df_val.loc[game_rows.index[1], 'Close Spread'] = np.abs(df_val.loc[game_rows.index[0], 'Close Spread'])

In [34]:
df_val[['game_id','ML', 'Open Spread', 'Close Spread']].head(6)

,game_id,ML,Open Spread,Close Spread
0,1,-155,-7.0,-3.0
1,1,155,7.0,3.0
2,2,-310,-6.5,-7.5
3,2,310,6.5,7.5
4,3,-135,-2.0,-2.5
5,3,135,2.0,2.5


In [35]:
def convert_moneyline_to_percentage(odds):
    if odds >= 0:
        return (100 / (odds + 100)) * 100
    else:
        return (odds / (odds - 100)) * 100

# Create a new column 'Percentage' and apply the conversion function
df_val['ML_pct'] = df_val['ML'].apply(convert_moneyline_to_percentage)

In [45]:
column_name = 'ML_pct'

# Get the list of column names in the current order
columns = list(df_val.columns)

# Remove the column to be moved from the list
columns.remove(column_name)

# Insert the column at the desired position (column number 6)
columns.insert(12, column_name)

# Reassign the column order to the dataframe
df_val = df_val[columns]

In [46]:
# df_val['ML_pct'] = df_val['ML_pct'].apply(lambda x: '{:.2f}'.format(round(x, 2)))

In [48]:
df_val

,game_id,Date,VH,Team,1st,2nd,3rd,4th,Final,Open,...,2H,Open Total,Close Total,Open Spread,Close Spread,W/L,W/L_bool,first_half,Score_Diff,ATS
0,1,1018,H,Boston,24,39,35,28,126,7.0,...,2,0.0,0.0,-7.0,-3.0,W,1,1,0.0,0
1,1,1018,V,Philadelphia,29,34,25,29,117,229.0,...,107,229.0,216.0,7.0,3.0,L,0,1,-9.0,0
2,2,1018,H,GoldenState,25,34,32,32,123,6.5,...,2,0.0,0.0,-6.5,-7.5,W,1,1,0.0,0
3,2,1018,V,LALakers,22,30,19,38,109,229.5,...,114,229.5,223.5,6.5,7.5,L,0,0,-14.0,0
4,3,1019,V,Washington,36,24,27,27,114,2.0,...,111.5,0.0,0.0,-2.0,-2.5,W,1,1,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,662,116,V,Utah,27,25,35,39,126,234.5,...,119,234.5,232.0,0.5,4.0,W,1,0,1.0,0
1324,663,116,H,Memphis,29,39,39,29,136,10.0,...,6.5,0.0,0.0,-10.0,-12.5,W,1,1,0.0,0
1325,663,116,V,Phoenix,36,29,23,18,106,225.5,...,118,225.5,231.0,10.0,12.5,L,0,0,-30.0,0
1326,664,116,H,LALakers,37,36,29,38,140,0.5,...,0,0.0,0.0,-0.5,-7.5,W,1,1,0.0,0


In [50]:
# Add Push col
df_val['Push'] = 0  # Initialize 'ATS_fav_less' column with zeros

for i in range(0, len(df_val), 2):
#     favorite_ml = df.loc[i, 'ML']
#     underdog_ml = df.loc[i + 1, 'ML']
    favorite_score = df_val.loc[i, 'Final']
    underdog_score = df_val.loc[i + 1, 'Final']
    close_spread = df_val.loc[i, 'Close Spread']
 
    if abs(favorite_score - underdog_score) == abs(close_spread):
        df_val.at[i, 'Push'] = 1

In [52]:
df_val[['game_id','ML', 'Open Spread', 'Close Spread']].head(6)

,game_id,ML,Open Spread,Close Spread
0,1,-155,-7.0,-3.0
1,1,155,7.0,3.0
2,2,-310,-6.5,-7.5
3,2,310,6.5,7.5
4,3,-135,-2.0,-2.5
5,3,135,2.0,2.5


In [53]:
#df_val.to_csv('../nss_capstone/df_val.csv')

In [56]:
pd.read_csv('../nss_capstone/df_val.csv').drop(columns = 'Unnamed: 0').fillna('')

,game_id,Date,VH,Team,1st,2nd,3rd,4th,Final,Open,...,Open Total,Close Total,Open Spread,Close Spread,W/L,W/L_bool,first_half,Score_Diff,ATS,Push
0,1,1018,H,Boston,24,39,35,28,126,7.0,...,0.0,0.0,-7.0,-3.0,W,1,1,0.0,0,0
1,1,1018,V,Philadelphia,29,34,25,29,117,229.0,...,229.0,216.0,7.0,3.0,L,0,1,-9.0,0,0
2,2,1018,H,GoldenState,25,34,32,32,123,6.5,...,0.0,0.0,-6.5,-7.5,W,1,1,0.0,0,0
3,2,1018,V,LALakers,22,30,19,38,109,229.5,...,229.5,223.5,6.5,7.5,L,0,0,-14.0,0,0
4,3,1019,V,Washington,36,24,27,27,114,2.0,...,0.0,0.0,-2.0,-2.5,W,1,1,7.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,662,116,V,Utah,27,25,35,39,126,234.5,...,234.5,232.0,0.5,4.0,W,1,0,1.0,0,0
1324,663,116,H,Memphis,29,39,39,29,136,10.0,...,0.0,0.0,-10.0,-12.5,W,1,1,0.0,0,0
1325,663,116,V,Phoenix,36,29,23,18,106,225.5,...,225.5,231.0,10.0,12.5,L,0,0,-30.0,0,0
1326,664,116,H,LALakers,37,36,29,38,140,0.5,...,0.0,0.0,-0.5,-7.5,W,1,1,0.0,0,0
